# Preliminary Work

### Problem 1(a)
For the mean squared error loss function $L = \frac{1}{2}(y - f(x))^2$, prove that $g(x) = -(y - f(x))$ and calculate $h(x)$.

##### Solution
We are given that,

$$g_i = \partial_{\hat{y_i}^{(t-1)}} L$$
$$h_i = \partial_{\hat{y_i}^{(t-1)}}^2 L$$

where,

$$ \hat{y_i}^{(t)} = \sum_{k=1}^{t} f_k(x_i) $$

For notational consistency between the problem statement and the xgboost documentation, set $f(x) = \hat{y}^{T}$ to be the sum of all the trees up to a given iteration $T$. 

\begin{align*}
g_i & = \partial_{f(x_i)} \frac{1}{2}(y_i - f(x_i))^2 \\
 & = (y_i - f(x_i))(-1) \\
 & = -(y_i - f(x_i)) .
\end{align*}

So $g(x) = -(y - f(x))$.

\begin{align*}
h_i & = \partial_{f(x_i)}^2 \frac{1}{2}(y_i - f(x_i))^2 \\
 & = -\partial_{f(x_i)} (y_i - f(x_i)) \\
 & = 1
\end{align*}

So $h(x) = 1$.

### Problem 1(b)
Assuming that the function $f_0(x) = 0$ for all $x$ and assuming that the data points $(x_i, y_i)$ all have $y_i$ equal to either $+1$ or $-1$, write out the corresponding functions $G$ and $H$ for the next tree, $f_1$. For a given proposed split of a leaf into leaves L and R, let

$l_p$ be the number of data points with $y_i=+1$ in the L leaf,

$l_m$ be the number of data points with $y_i=-1$ in the L leaf,

$r_p$ be the number of data points with $y_i=+1$ in the R leaf,

$r_m$ be the number of data points with $y_i=-1$ in the R leaf.

Show that the gain function used for splitting a tree in xgboost in this special case is 

$$Gain = \frac{(l_p - l_m)^2}{2(l_p + l_m + \lambda)}  + \frac{(r_p - r_m)^2}{2(r_p + r_m + \lambda)} - \frac{(l_p + r_p - l_m - r_m)^2}{2(l_p + r_p + l_m + r_m + \lambda)} - \gamma$$


##### Solution

In the xgboost documentation, we are given that,

$$Gain = \frac{1}{2} \left[\frac{G_L^2}{H_L+\lambda}+\frac{G_R^2}{H_R+\lambda}-\frac{(G_L+G_R)^2}{H_L+H_R+\lambda}\right] - \gamma$$

This problem, therefore, reduces to showing that,
\begin{align*}
G_L & = \pm (l_p - l_m) \\
G_R & = \pm (r_p - r_m) \\
H_L & = l_p + l_m \\
H_R & = r_p + r_m 
\end{align*}

By definition given in the documentation,
\begin{align*}
G_L & = \sum_{i \in I_L} g_i \\
 & = \sum_{y_i = +1} g_i + \sum_{y_i = -1} g_i \\
 & = \sum_{y_i = +1} -1 + f_0(x_i) + \sum_{y_i = -1} 1 + f_0(x_i) \\
 & = 2\sum_{i} f(x_i) + \sum_{y_i = +1} -1 + \sum_{y_i = -1} 1 \\
 & = \sum_{y_i = +1} -1 + \sum_{y_i = -1} 1 \\
 & = -l_p + l_m \\
 & = -(l_p - l_m)
\end{align*}

$G_R = -(r_p - r_m)$ follows similarly.

By defintion given in the documentation, 
\begin{align*}
H_L & = \sum{i \in I_L} h_i \\
 & = \sum_{y_i = +1} h_i + \sum_{y_i = -1} h_i \\
 & = \sum_{y_i = +1} 1 + \sum_{y_i = -1} 1 \\
 & = l_p + l_m
\end{align*}

$H_R = r_p + r_m$ follows similarly.   QED

### Problem 2
Do the same computations for the adaboost loss function $L(y,f(x)) = e^{-yf(x)}.$ 

##### Solution.
For brevity, I will not reintroduce notation and I will not be as thorough in presenting all my work.
$$L = e^{-yf(x)}$$
Therefore, 
$$g = -ye^{-yf(x)}$$
$$h = y^2e^{-yf(x)}$$.

\begin{align*}
G_L & = \sum_{i \in I_L} g_i \\
 & = \sum_{y_i = +1} g_i + \sum_{y_i = -1} g_i \\
 & = \sum_{y_i = +1} -e^{-f_0(x_i)} + \sum_{y_i = -1} e^{f_0(x_i)} \\
 & = \sum_{y_i = +1} -1 + \sum_{y_i = -1} 1 \\
 & = -l_p + l_m \\
 & = -(l_p - l_m)
\end{align*}

$G_R = -(r_p - r_m)$ follows similarly.

\begin{align*}
H_L & = \sum{i \in I_L} h_i \\
 & = \sum_{y_i = +1} h_i + \sum_{y_i = -1} h_i \\
 & = \sum_{y_i = +1} y_i^2e^{-y_if_0(x)} + \sum_{y_i = -1} y_i^2e^{-y_if(x)} \\
 & = \sum_{y_i = +1} (1)^2e^{-f_0(x)} + \sum_{y_i = -1} (-1)^2e^{f_0(x)} \\
 & = \sum_{y_i = +1} 1 + \sum_{y_i = -1} 1 \\
 & = l_p + l_m
\end{align*}

$H_R = r_p + r_m$ follows similarly.   QED

# Utilizing the XGBoost Algorithm
__NOTE:__ due to the grid searching used to tune the parameters, it is not in your best interest to rerun this code.

In [219]:
from __future__ import division
import numpy as np
import pandas as pd
import time
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV

import xgboost as xgb
from xgboost.sklearn import XGBClassifier

### Load the Data

In [2]:
breast_cancer = load_breast_cancer()
data = pd.DataFrame(breast_cancer["data"], columns=breast_cancer["feature_names"])
target = breast_cancer["target"]

In [3]:
data.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [4]:
def test_model(tree, features, target, iters=20):
    """
    Test a decision tree model based on inputed data and target.
    Print the average misclassification rate, variance, and 
    average training time of the model.
    
    Inputs:
        tree - sklearn DecisionTreeClassifier object
        features - features of the data
        target - target values for the data
        iters (optional) - number of iterations to test model. 
            Defaults to 50.
    """
    acc = []
    t = []
    for i in xrange(iters):
        # create a new train-test split each iteration
        Xtrain, Xtest, ytrain, ytest = train_test_split(features, target, train_size=.7)
        
        # fit and time
        before = time.time()
        tree.fit(Xtrain, ytrain)
        after = time.time()
        t.append(after-before)
        
        # calculate and store misclassification rate
        acc.append(1 - (tree.predict(Xtest) == ytest).mean())
    print "Average misclassification rate:", np.mean(acc)
    print "Variance of misclassification rate:", np.var(acc)
    print "Average training time:", np.mean(t)

In [ ]:
def report(results, n_top=3):
    """
    Print the 'n_top' best model parameters and performance.
    """
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

# Tuning xgboost parameters

### Vanilla xgboost

In [5]:
xgb = XGBClassifier()
test_model(xgb, data, target)

Average misclassification rate: 0.0388888888889
Variance of misclassification rate: 0.000161673677371
Average training time: 0.0661438107491


In [6]:
xgb1 = XGBClassifier(max_depth=4)
test_model(xgb1, data, target)

Average misclassification rate: 0.0397660818713
Variance of misclassification rate: 0.000169624841832
Average training time: 0.0757920622826


In [7]:
xgb2 = XGBClassifier(min_child_weight=.8)
test_model(xgb2, data, target)

Average misclassification rate: 0.0362573099415
Variance of misclassification rate: 0.00016620498615
Average training time: 0.0688552618027


In [8]:
xgb3 = XGBClassifier(learning_rate=.2)
test_model(xgb3, data, target)

Average misclassification rate: 0.040350877193
Variance of misclassification rate: 0.000280086180363
Average training time: 0.05118714571


In [9]:
xgb4 = XGBClassifier(n_estimators=120)
test_model(xgb4, data, target)

Average misclassification rate: 0.0377192982456
Variance of misclassification rate: 0.000295732020109
Average training time: 0.0739785313606


In [14]:
xgb5 = XGBClassifier(subsample=.7, colsample_bytree=.3)
test_model(xgb5, data, target)

Average misclassification rate: 0.030701754386
Variance of misclassification rate: 0.000156885879416
Average training time: 0.0261357426643


### Grid Search for Parameters
For my 6th xgboost model, I will use a step-by-step parameter grid search to find the best model I can. 

I will determine the optimal parameter values based on the subset of parameters I'm considering at each step. At each step, I will add a few more parameters and repeat the process until I have searched a variety of values for all parameters.

#### learning_rate and n_estimators

In [35]:
param_grid1 = {
    'objective': ['binary:logistic'],
    'learning_rate': np.arange(0.05,0.25,.025),
    'silent': [1.0],
    'n_estimators': np.arange(100,200,10)
}

grid_search = GridSearchCV(XGBClassifier(), param_grid=param_grid1)
fit = grid_search.fit(data, target)

report(grid_search.cv_results_, n_top=1)

#### max_depth and min_child_weight

In [41]:
param_grid2 = {
    'objective': ['binary:logistic'],
    'max_depth': [3,4,5,6], #4
    'min_child_weight': np.arange(.5,1.5,.1), #1
    'learning_rate': [.2],
    'silent': [1.0], 
    'n_estimators': [120]
}

grid_search = GridSearchCV(XGBClassifier(), param_grid=param_grid2)
fit = grid_search.fit(data, target)

report(grid_search.cv_results_, n_top=1)

#### gamma

In [101]:
param_grid3 = {
    'objective': ['binary:logistic'],
    'max_depth': [4],
    'min_child_weight': [1.3],
    'learning_rate': [.2],
    'silent': [1.0], 
    'n_estimators': [120],
    'gamma':np.arange(0,1,.1)
}

grid_search = GridSearchCV(XGBClassifier(), param_grid=param_grid3)
fit = grid_search.fit(data, target)

report(grid_search.cv_results_, n_top=1)

Model with rank: 1
Mean validation score: 0.974 (std: 0.004)
Parameters: {'silent': 1.0, 'learning_rate': 0.2, 'min_child_weight': 1.3, 'n_estimators': 120, 'objective': 'binary:logistic', 'max_depth': 4, 'gamma': 0.0}



#### subsample and colsample_bytree

In [103]:
param_grid4 = {
    'objective': ['binary:logistic'],
    'max_depth': [4],
    'min_child_weight': [1.3],
    'learning_rate': [.2],
    'silent': [1.0], 
    'n_estimators': [120],
    'gamma':[0],
    'subsample':np.arange(0.1,1,.1),
    'colsample_bytree':np.arange(0.1,1,.1)
}

grid_search = GridSearchCV(XGBClassifier(), param_grid=param_grid4)
fit = grid_search.fit(data, target)

report(grid_search.cv_results_, n_top=1)

Model with rank: 1
Mean validation score: 0.981 (std: 0.010)
Parameters: {'colsample_bytree': 0.30000000000000004, 'silent': 1.0, 'learning_rate': 0.2, 'min_child_weight': 1.3, 'n_estimators': 120, 'subsample': 0.90000000000000002, 'objective': 'binary:logistic', 'max_depth': 4, 'gamma': 0}

Model with rank: 1
Mean validation score: 0.981 (std: 0.007)
Parameters: {'colsample_bytree': 0.40000000000000002, 'silent': 1.0, 'learning_rate': 0.2, 'min_child_weight': 1.3, 'n_estimators': 120, 'subsample': 0.70000000000000007, 'objective': 'binary:logistic', 'max_depth': 4, 'gamma': 0}



#### reg_alpha and reg_lambda

In [61]:
param_grid5 = {
    'objective': ['binary:logistic'],
    'max_depth': [4],
    'min_child_weight': [1.3],
    'learning_rate': [.2],
    'silent': [1.0], 
    'n_estimators': [120],
    'reg_alpha':[10**t for t in [-10, -5, -3, -2, -1, 0]], #0.001
    'reg_lambda':np.arange(0,1,.1), #0.8
    'gamma':[0],
    'subsample': [0.7],
    'colsample_bytree': [0.4]
}

grid_search = GridSearchCV(XGBClassifier(), param_grid=param_grid5)
fit = grid_search.fit(data, target)

report(grid_search.cv_results_, n_top=1)

#### scale_pos_weight

In [66]:
param_grid6 = {
    'objective': ['binary:logistic'],
    'max_depth': [4],
    'min_child_weight': [1.3],
    'learning_rate': [.2],
    'silent': [1.0], 
    'n_estimators': [120],
    'reg_alpha': [0.001], #0.001
    'reg_lambda': [0.6], #0.8
    'gamma':[0],
    'subsample': [0.7],
    'colsample_bytree': [0.4],
    'scale_pos_weight':np.arange(0,1.25,.1)
}

grid_search = GridSearchCV(XGBClassifier(), param_grid=param_grid6)
fit = grid_search.fit(data, target)

report(grid_search.cv_results_, n_top=1)

Model with rank: 1
Mean validation score: 0.981 (std: 0.007)
Parameters: {'reg_alpha': 0.001, 'colsample_bytree': 0.4, 'silent': 1.0, 'scale_pos_weight': 1.0, 'learning_rate': 0.2, 'min_child_weight': 1.3, 'n_estimators': 120, 'subsample': 0.7, 'reg_lambda': 0.6, 'objective': 'binary:logistic', 'max_depth': 4, 'gamma': 0}



### Final model

In [79]:
xgb_params = {
    'objective': 'binary:logistic',
    'max_depth': 4,
    'min_child_weight': 1.3,
    'learning_rate': .2,
    'silent': 1,
    'n_estimators': 120,
    'reg_alpha': 0.001,
    'reg_lambda': 0.8,
    'gamma': 0,
    'subsample': 0.7,
    'colsample_bytree': 0.4,
    'scale_pos_weight': 1
}

best_xgb = XGBClassifier(**xgb_params)
test_model(best_xgb, data, target)

Average misclassification rate: 0.0321637426901
Variance of misclassification rate: 0.000100885742622
Average training time: 0.0297340989113


# Model Comparison
I will compare my refined xgboost model to default decision trees, default random forests, and default gradient boosted trees. I will also compare my refined xgboost model to the best models from my previous homework assignments.

In [216]:
tree = DecisionTreeClassifier()
rf = RandomForestClassifier()
gb = GradientBoostingClassifier()
xgb = XGBClassifier()

gb_params = {'loss': 'exponential', 'learning_rate': 0.5, 
               'n_estimators': 100, 'min_samples_split': 6, 
               'max_features': 15, 'max_depth': 2}

xgb_params = {
        'objective': 'binary:logistic',
        'max_depth': 4,
        'min_child_weight': 1.3,
        'learning_rate': .2,
        'silent': 1,
        'n_estimators': 120,
        'reg_alpha': 0.001,
        'reg_lambda': 0.8,
        'gamma': 0,
        'subsample': 0.7,
        'colsample_bytree': 0.4,
        'scale_pos_weight': 1
    }

best_tree = DecisionTreeClassifier(criterion='entropy', splitter="random", min_samples_leaf=5)
best_rf = RandomForestClassifier(n_estimators=50, criterion="entropy", min_samples_split=4)
best_gb = GradientBoostingClassifier(**gb_params)
best_xgb = XGBClassifier(**xgb_params)

tree_times = []
rf_times = []
gb_times = []
xgb_times = []
best_tree_times = []
best_rf_times = []
best_gb_times = []
best_xgb_times = []

tree_acc = []
rf_acc = []
gb_acc = []
xgb_acc = []
best_tree_acc = []
best_rf_acc = []
best_gb_acc = []
best_xgb_acc = []

for i in xrange(100):
    Xtrain, Xtest, ytrain, ytest = train_test_split(data, target, train_size=.7)
    
    before = time.time()
    tree.fit(Xtrain, ytrain)
    after = time.time()
    tree_times.append(after - before)
    tree_acc.append(1 - (tree.predict(Xtest) == ytest).mean()) 
    
    before = time.time()
    rf.fit(Xtrain, ytrain)
    after = time.time()
    rf_times.append(after - before)
    rf_acc.append(1 - (rf.predict(Xtest) == ytest).mean()) 
    
    before = time.time()
    gb.fit(Xtrain, ytrain)
    after = time.time()
    gb_times.append(after - before)
    gb_acc.append(1 - (gb.predict(Xtest) == ytest).mean()) 

    before = time.time()
    xgb.fit(Xtrain, ytrain)
    after = time.time()
    xgb_times.append(after - before)
    xgb_acc.append(1 - (xgb.predict(Xtest) == ytest).mean()) 
    
    before = time.time()
    best_tree.fit(Xtrain, ytrain)
    after = time.time()
    best_tree_times.append(after - before)
    best_tree_acc.append(1 - (best_tree.predict(Xtest) == ytest).mean()) 
    
    before = time.time()
    best_rf.fit(Xtrain, ytrain)
    after = time.time()
    best_rf_times.append(after - before)
    best_rf_acc.append(1 - (best_rf.predict(Xtest) == ytest).mean()) 
    
    before = time.time()
    best_gb.fit(Xtrain, ytrain)
    after = time.time()
    best_gb_times.append(after - before)
    best_gb_acc.append(1 - (best_gb.predict(Xtest) == ytest).mean()) 
    
    before = time.time()
    best_xgb.fit(Xtrain, ytrain)
    after = time.time()
    best_xgb_times.append(after - before)
    best_xgb_acc.append(1 - (best_xgb.predict(Xtest) == ytest).mean()) 
    
print "Misclassification Rates:"
print "------------------------"
print "  Default Tree:\t", np.average(tree_acc)
print "  Best Tree:\t", np.average(best_tree_acc)
print ""
print "  Default RF:\t", np.average(rf_acc)
print "  Best RF:\t", np.average(best_rf_acc)
print ""
print "  Default GB:\t", np.average(gb_acc)
print "  Best GB:\t", np.average(best_gb_acc)
print ""
print "  Default XGB:\t", np.average(xgb_acc)
print "  Best XGB:\t", np.average(best_xgb_acc)
print ""
print "Training Times:"
print "---------------"
print "  Tree Training Time:\t", np.average(best_tree_times)
print "  RF Training Time:\t", np.average(best_rf_times)
print "  GB Training Time:\t", np.average(best_gb_times)
print "  XGB Training Time:\t", np.average(best_xgb_times)

Misclassification Rates:
------------------------
  Default Tree:	0.075730994152
  Best Tree:	0.0661988304094

  Default RF:	0.0488888888889
  Best RF:	0.043918128655

  Default GB:	0.0452631578947
  Best GB:	0.036081871345

  Default XGB:	0.0391228070175
  Best XGB:	0.0338011695906

Training Times:
---------------
  Tree Training Time:	0.000704345703125
  RF Training Time:	0.199412851334
  GB Training Time:	0.0591463565826
  XGB Training Time:	0.0310510110855


As expected, the refined models outperformed their default counterparts. 

I was surprised to see how fast xgboost was in comparison to regular gradient boosting--nearly twice as fast. This was especially surprising since the n_estimators=100, max_depth=2 on the gradient boosted trees whereas n_estimators=120, max_depth=4 on xgboost. I am unsure if this is a difference in the quality of the implemations or if it due to the algorithms itself. As another comparison, unsurprisingly, decision trees were by far the fastest.

It was interesting to see that xgboost did not experience a major improvement over gradient boosted trees. I think this is a testament to how similar they are. The extra care given to regularization is what gives xgboost the edge.